# OpenAI results retrieval 

Retrieve OpenAI batch scripts once they are done running.

In [1]:
%pip install openai dotenv pandas

  Using cached openai-1.76.0-py3-none-any.whl.metadata (25 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.9.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-no

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

batch_id = "batch_680b3986ee4c8190a09280a652461ee1"

batch = client.batches.retrieve(batch_id)
print(batch.status) # Check status. Should be 'completed'

completed


Once completed, retrieve with:

In [ ]:
file_id = batch.output_file_id
file_response = client.files.content(file_id=file_id)
print(file_response.text.splitlines()[0])

{"id": "batch_req_680b39d653948190860bfb36c3ad5c29", "custom_id": "0", "response": {"status_code": 200, "request_id": "d50340c6a7d007cf00b30186abb9c6de", "body": {"id": "chatcmpl-BQ7lr4YHn2T5Cm0NUff5GMsQ1lHOP", "object": "chat.completion", "created": 1745566099, "model": "gpt-4.1-2025-04-14", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\"count\":8}", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 486, "completion_tokens": 6, "total_tokens": 492, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_b38e740b47"}}, "error": null}


Save as DataFrame

In [39]:
import json
import pandas as pd

jsonified = [
    json.loads(line)
    for line in file_response.text.splitlines()
]

custom_ids = [
    obj['custom_id']
    for obj in jsonified
]

responses = [
    json.loads(
        obj['response']['body']['choices'][0]['message']['content']        
    )['count']
    for obj in jsonified
]

df = pd.DataFrame({
    'custom_id': custom_ids,
    'response': responses
})
df.head()

,custom_id,response
0,0,8
1,1,11
2,2,10
3,3,81
4,4,9


Save as csv:

In [ ]:
import time

timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
import os

DATASET='TallyQA'
DIR = '../eval/results/openai'

model = jsonified[0]['response']['body']['model'].replace("/", "_")

os.makedirs(DIR + '/' + model, exist_ok=True)
df.to_csv(f"{DIR}/{model}/{timestamp}/{DATASET}.csv", index=False)

print(f"Results saved to {DIR}/{model}/{timestamp}/{DATASET}.csv")

gpt-4.1-2025-04-14
